In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('Graphs-HW2').config("spark.sql.crossJoin.enabled", True).getOrCreate()
from pyspark.sql.types import *

pr_sdf = spark.read.load('pr_graph.txt', format="text")
pr_sdf = pr_sdf.select(F.split(pr_sdf.value, ' ')[0].alias('from_node').cast('int'),
                            F.split(pr_sdf.value, ' ')[1].alias('to_node').cast('int'),
                            F.split(pr_sdf.value, ' ')[2].alias('timestamp').cast('int'))

In [8]:
def pagerank(G, num_iter):
    G.createOrReplaceTempView('G')
    h1 = spark.sql('''SELECT DISTINCT from_node, ROW_NUMBER() OVER(ORDER BY from_node) AS count
                FROM G
                GROUP BY from_node''')

    h1.createOrReplaceTempView('h1')
    m = spark.sql('''SELECT MAX(count) AS max
                FROM h1''')

    N = 1/m.head()[0]

    h1 = G.select(G.from_node).withColumn('proportion',F.lit(N))
    h1.createOrReplaceTempView('h1')
    h1 = spark.sql('''SELECT DISTINCT from_node, proportion
                FROM h1''')

    h2 = G.groupBy(G['from_node']).count()
    h1.createOrReplaceTempView('h1')
    h2.createOrReplaceTempView('h2')
    h2 = spark.sql('''SELECT h1.from_node AS from_node1, 1/(h2.count) as proportion
                FROM h1 JOIN h2
                ON h1.from_node = h2.from_node''')

    G.createOrReplaceTempView('G')
    h2.createOrReplaceTempView('h2')
    h3 = spark.sql('''SELECT DISTINCT p.from_node as from_node1, p.to_node AS node, h2.proportion AS pagerank
                FROM G p JOIN h2
                WHERE p.from_node = h2.from_node1
                ORDER BY node''')

    for i in range(0, num_iter):
        h4 = h1.join(h3, h1.from_node == h3.from_node1)
        h4 = h4.selectExpr('from_node', 'node', 'proportion', 'pagerank', 'proportion*pagerank')

        h4.createOrReplaceTempView('h4')
        h5 = spark.sql('''SELECT DISTINCT node as from_node, 0.85 * sum(proportion*pagerank) + 0.15 AS proportion
                FROM h4
                GROUP BY node''')
        h1 = h5
    return h5

# Step 5

In [3]:
pr_sdf.count()

16

In [4]:
pr_sdf.show()

+---------+-------+---------+
|from_node|to_node|timestamp|
+---------+-------+---------+
|        1|      2|        0|
|        1|      3|        0|
|        1|      4|        0|
|        1|      5|        0|
|        2|      3|        0|
|        2|      5|        0|
|        3|      2|        0|
|        4|      5|        0|
|        5|      1|        0|
|        5|      6|        0|
|        5|      7|        0|
|        6|      7|        0|
|        7|      6|        0|
|        7|      2|        0|
|        7|      7|        0|
|        5|      4|        0|
+---------+-------+---------+



In [9]:
pagerank(pr_sdf, 5).show()

+---------+-------------------+
|from_node|         proportion|
+---------+-------------------+
|        4|0.36610546184882914|
|        1| 0.3056855115440409|
|        5| 0.8147069488207835|
|        3| 0.5262470701160463|
|        6| 0.5380753784802238|
|        2|  0.843575161873236|
|        7| 0.9433725923168398|
+---------+-------------------+

